# Investments Project (Spring 2024)

**Authors:**
- Marc-Antoine Allard
- Adam Zinebi
- Paul Teiletche
- ...

**DUE Date: June 21 at 23:59**

---
# Imports

In [1]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import datetime
#import wrds
import seaborn as sns

---
# Idiosyncratic Volatility Strategy (IV)

(a) Compute the time-varying estimate for each stock’s idiosyncratic volatility σidio
t,n ob-
tained as the volatility of the residual in the monthly rolling 5-year regressions of
stock-specific excess returns on the excess market return. Require at least 36 months
of observations for each stock. Winsorize the volatility at 5 and 95 %.

In [ ]:
...

(b) At every month t, sort all stocks into deciles based on their idiosyncratic volatility
(estimated using the most recent rolling window). Then compute monthly returns for
10 decile portfolios that equal weight all stocks in each decile. Plot the average annu-
alized portfolio mean, standard deviation, and Sharpe ratios across the 10 deciles in
three barplots. Repeat for value-weighted decile portfolios. Summarize your findings.
Is the evidence consistent with the CAPM?

In [ ]:
...

(c) Now we construct the idiosyncratic volatility factor. At every month t, we go long
the three highest decile volatility portfolios and we go short the three lowest decile
volatility portfolios. Compute and compare the mean, standard deviation, and Sharpe
ratios of the long and short legs of the strategy as well as of the strategy itself. Test if
the strategy has an average return that is statistically significantly different from zero.
Repeat both tests for equal and value-weighted portfolios. How do your results differ
from Ang, Hodrick, Xing, and Zhang (2006; table VI page 285) and what may be the
explanation for the difference?

In [ ]:
...